# Уровень потребительской лояльности


ОПИСАНИЕ: Заказчик этого исследования — крупная телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. Перед компанией стоит ***задача*** определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России.  
  
Чтобы определить уровень лояльности, клиентам задавали классический вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым».

ИССЛЕДУЕМЫЕ ВОПРОСЫ: 
- Как распределены участники опроса по возрасту и полу? Каких пользователей больше: новых или старых? Пользователи из каких городов активнее участвовали в опросе?
- Какие группы пользователей наиболее лояльны к сервису? Какие менее?
- Какой общий NPS среди всех опрошенных?
- Как можно описать клиентов, которые относятся к группе cторонников (англ. promoters)?

Ответственный за проект: Волков Владимир Александрович  
Контактные данные: Vladimir.Volkov@gmail.com  
Рабочий телефон: 8 977 888-77-71  

Первым этапом проекта будет выгрузка данных из БД

In [6]:
# загрузка необходимых библиотек
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

In [3]:
# Запрос на выгрузку данных из sqlite
query = """
SELECT 
    u.user_id as user_id 
    ,u.lt_day as lt_day
    ,CASE 
        WHEN u.lt_day <= 365 THEN True
        ELSE False
    END as is_new
    ,u.age as age
    ,CASE 
          WHEN u.gender_segment = 1 THEN 'Женский'
          ELSE 'Мужской'
    END as gender_segment
    ,u.os_name as os_name
    ,u.cpe_type_name as cpe_type_name
    ,l.country as country
    ,l.city as city
    ,a.title as age_segment
    ,ts.title as traffic_segment
    ,ls.title as lifetime_segment
    ,u.nps_score as nps_score
    ,CASE 
        WHEN u.nps_score >= 9 THEN 'сторонники'
        WHEN u.nps_score >= 7 THEN 'нейтралы'
        WHEN u.nps_score >= 0 THEN 'критики'
    END as nps_group
FROM user u
jOIN location l ON l.location_id = u.location_id
JOIN age_segment a ON a.age_gr_id = u.age_gr_id
JOIN traffic_segment ts ON ts.tr_gr_id = u.tr_gr_id
JOIN lifetime_segment ls ON ls.lt_gr_id = u.lt_gr_id
"""

In [4]:
df = pd.read_sql(query, engine)
df.head(10)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,0,45.0,Женский,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,04 1-5,08 36+,10,сторонники
1,A001WF,2344,0,53.0,Мужской,ANDROID,SMARTPHONE,Россия,Киров,05 45-54,04 1-5,08 36+,10,сторонники
2,A003Q7,467,0,57.0,Мужской,ANDROID,SMARTPHONE,Россия,Москва,06 55-64,08 20-25,06 13-24,10,сторонники
3,A004TB,4190,0,44.0,Женский,IOS,SMARTPHONE,Россия,РостовнаДону,04 35-44,03 0.1-1,08 36+,10,сторонники
4,A004XT,1163,0,24.0,Мужской,ANDROID,SMARTPHONE,Россия,Рязань,02 16-24,05 5-10,08 36+,10,сторонники
5,A005O0,5501,0,42.0,Женский,ANDROID,SMARTPHONE,Россия,Омск,04 35-44,05 5-10,08 36+,6,критики
6,A0061R,1236,0,45.0,Мужской,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,06 10-15,08 36+,10,сторонники
7,A009KS,313,1,35.0,Мужской,ANDROID,SMARTPHONE,Россия,Москва,04 35-44,13 45-50,05 7-12,10,сторонники
8,A00AES,3238,0,36.0,Женский,ANDROID,SMARTPHONE,Россия,СанктПетербург,04 35-44,04 1-5,08 36+,10,сторонники
9,A00F70,4479,0,54.0,Женский,ANDROID,SMARTPHONE,Россия,Волгоград,05 45-54,07 15-20,08 36+,9,сторонники


Данные выгрузились корректно. Группировки проставили верно

In [7]:
# Сохранение итоговой таблицы в файл
df.to_csv('telecomm_csi_tableau.csv', index=False)

Второй этап, создание дашборда
https://public.tableau.com/views/Project_2_17301219157060/Story1?:language=en-US&publish=yes&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link